## Overview

### Structure

The pipeline is as follows:

Download the city gml files -> Convert to shapefiles -> Divide into grids -> Calculate UMP for each grid -> Save as y

Loop through each grid, download sentinel imagery, store and write as tensor 

### Datasets

- X:
    - Sentinel
- Y:
    - Tokyo (Japan, 2021) https://www.geospatial.jp/ckan/dataset/plateau-tokyo23ku/resource/0bab2b7f-6962-41c8-872f-66ad9b40dcb1?inner_span=True
    - Osaka (Japan, 2021) ^ 
    - New York (USA, 2019) https://github.com/opencitymodel/opencitymodel 

## Import Libraries

In [45]:
import os.path
from multiprocessing import Pool

from itertools import repeat
from glob import glob
import xml.dom.minidom

import geopandas as gpd
import pandas as pd
import numpy as np

import fiona
import shapely
import pyproj


## Convert GML to shp

### Function Definitions

Convert all of the GML files in a folder into a single shapefile

In [92]:
def xml_extract_gdf(in_path, 
    building_tag= "bldg:Building", 
    lod_tag= "bldg:lod1Solid", 
    polygon_tag= "gml:Polygon", 
    coord_tag= "gml:posList", 
    height_tag= "bldg:measuredHeight", 
    src_crs= "EPSG:6668", 
    tgt_crs= "EPSG:3857"):
    """
    Extracts the building geometry (first polygon of each building only) and height, and assemble them into a GeoDataFrame\n
    # Parameters:\n
    - in_path: File path to the input gml file\n
    - building_tag: The tag for each building, number of tags should be equivalent to the number of buildings\n
    - lod_tag: The tag specifying which lod to extract from\n
    - polygon_tag: The tag for polygons making up each building, only the first polygon will be extracted\n
    - coord_tag: The tag for the list of coordinates\n
    - height_tag: The tag for the height, should have same number of instances as buildings\n
    """
    # Manually read the gml as xml and extract the geometry
    doc = xml.dom.minidom.parse(in_path)
    buildings = doc.getElementsByTagName("bldg:lod1Solid") # This is Plateau dataset specific
    buildings_coords = [building\
                        .getElementsByTagName("gml:Polygon")[0]\
                        .getElementsByTagName("gml:posList")[0]\
                        .childNodes[0]\
                        .nodeValue.split(" ") for building in buildings]

    # Abort if no geometry found
    if len(buildings_coords) == 0:
        return None

    # Conversion is done here to preserve the precision due to python rounding shennanigans
    transformer = pyproj.Transformer.from_crs(src_crs, tgt_crs)
    buildings_coords = [
                        [transformer.transform(buildings_coords[bld][i], 
                            buildings_coords[bld][i+1]) for i in range(0, len(buildings_coords[bld]), 3)] 
                        for bld in range(len(buildings_coords))
                        ]
    geometry = gpd.GeoSeries([shapely.Polygon(coords) for coords in buildings_coords])
    
    # Manually extract the height
    buildings_height = doc.getElementsByTagName("bldg:measuredHeight")

    # Abort if no height data found
    if len(buildings_height) == 0:
        return None

    buildings_height = [float(height.childNodes[0].nodeValue) for height in buildings_height]
    height_series = pd.DataFrame(buildings_height, columns= ["height"])

    gdf = gpd.GeoDataFrame(data= height_series, geometry= geometry)
    return gdf

def gml_to_feather(in_path, out_path, mode= None, log_name= "gml_convert", src_crs= "EPSG:6668", tgt_crs= "EPSG:3857"):
    """
    Takes in a gml file and outputs it as a feather file\n
    W/R with feather files is much faster and takes up much less space than using shp files\n
    # Parameters:\n
    - in_path: The path for the gml file\n
    - out_path: The output path for the shape file, must end with a .shp\n
    - mode: 
        - 'o' = overwrites any file at output path, \n
        - None = raises error if file already exists\n
    - src_crs: Source projection\n
    - tgt_crs: Target projection\n
    """
    # Extracts features
    with fiona.open(in_path, 'r') as src:
        features = list(src)

    # Converts and places it in geopandas format
    # There seems to be some gml files without the measured height column, will try to log those files in
    gdf = gpd.GeoDataFrame.from_features(features)
    try:
        gdf = gdf[['measuredHeight', 'geometry']]
        gdf.rename(columns={'measuredHeight':'height'}, inplace= True)

        # Remove the NaN values
        gdf = gdf.dropna().reset_index(drop= True)

        # Covert it to correct projection and strip to polygon instead from multi polygon
        gdf = gdf.explode(index_parts= True).set_crs(src_crs).to_crs(tgt_crs).loc[(slice(None), slice(0)), :].reset_index(drop= True)

        # Convert coordinates from 2D to 3D
        gdf_geometry = gpd.GeoSeries.from_wkb(gdf.to_wkb(output_dimension= 2)["geometry"])
        gdf.drop(["geometry"], axis= 1, inplace= True)
        gdf = gpd.GeoDataFrame(gdf, geometry= gdf_geometry)
    except Exception as e:
        # If exception occurs try to extract manually
        gdf = xml_extract_gdf(in_path)

        if gdf is None:
            print(f"{e}: {os.path.basename(in_path)}")
            if not log_name is None:
                if not os.path.exists("logs"):
                    os.makedirs("logs")
                with open(f"logs/{log_name}.txt", "a") as f:
                    f.write(in_path + "\n")
            return len(gdf)

    # Check if parent directory exists
    if not os.path.exists(os.path.dirname(out_path)):
        os.makedirs(os.path.dirname(out_path))
        
    # Outputs to the desired path
    if os.path.exists(out_path):
        if mode == "a":
            gdf.to_feather(out_path, mode= "a")
        elif mode == "o":
            gdf.to_feather(out_path)
        else:
            raise FileExistsError("Output path already exists")
    else:
        gdf.to_feather(out_path)
    
    return 0

def batch_gml_to_feather(in_dir, out_path, n_processes= 12, log_name= None, mode= None, src_crs= "EPSG:6668", tgt_crs= "EPSG:3857"):

    # Get all the paths of the gml files
    in_paths = glob(f"{in_dir}/*.gml")
    print("Total input files:", len(in_paths))

    # Reads the gml file and extract features
    with Pool(processes= n_processes) as pool:
        r = pool.starmap(
            gml_to_feather, 
            zip(in_paths, 
                [f'{in_dir}/temp/{os.path.basename(path).replace(".gml", ".feather")}' for path in in_paths], 
                repeat(mode), 
                repeat(log_name),
                repeat(src_crs),
                repeat(tgt_crs)))

    # Check for invalid buildings
    print(f"There are {sum(r)} invalid buildings from {len(list(filter(lambda x: x > 0, r)))} files")

    # Get all the paths of the shp files
    in_paths = glob(f"{in_dir}/temp/*.feather")
    print("Total files to merge:", len(in_paths))

    gdfs = [gpd.read_feather(in_path) for in_path in in_paths]
    gdf = gpd.GeoDataFrame(pd.concat(gdfs)).reset_index(drop= True)
    gdf.to_feather(out_path)

    for temp_file in in_paths:
        os.remove(temp_file)

    return gdf

### Tokyo

In [93]:
in_dir = "data/13100_tokyo23-ku_2020_citygml_3_2_op/udx/bldg"
out_path = "data/full_Tokyo_plateau/tokyo_fixed.feather"

batch_gml_to_feather(in_dir, out_path, mode= "o", log_name= "tokyo")

Total input files: 671
There are 0 invalid buildings from 0 files
Total files to merge: 671


,height,geometry
0,6.1,"POLYGON ((15565422.798 4245286.909, 15565416.2..."
1,3.0,"POLYGON ((15565401.386 4245273.087, 15565398.9..."
2,3.5,"POLYGON ((15563817.297 4264833.695, 15563811.6..."
3,11.8,"POLYGON ((15563065.418 4265104.495, 15563064.8..."
4,2.5,"POLYGON ((15563458.540 4264729.005, 15563431.3..."
...,...,...
1740782,12.6,"POLYGON ((15562651.930 4267173.365, 15562649.0..."
1740783,3.0,"POLYGON ((15563045.332 4266687.372, 15563043.5..."
1740784,9.2,"POLYGON ((15563072.667 4266950.506, 15563071.4..."
1740785,6.8,"POLYGON ((15562830.893 4267045.363, 15562827.3..."


### Osaka

In [7]:
in_dir = "data/osaka/udx/bldg"
out_path = "data/osaka/osaka_full.feather"

batch_gml_to_feather(in_dir, out_path, mode= "o", log_name= "osaka")

Total input files: 269
"['measuredHeight'] not in index": 51357370_bldg_6697_op.gml
"['measuredHeight'] not in index": 52350389_bldg_6697_op.gml
"['measuredHeight'] not in index": 52350378_bldg_6697_op.gml
"['measuredHeight'] not in index": 52350379_bldg_6697_op.gml
"['measuredHeight'] not in index": 52350368_bldg_6697_op.gml
"['measuredHeight'] not in index": 52350422_bldg_6697_op.gml
"['measuredHeight'] not in index": 52350480_bldg_6697_op.gml
"['measuredHeight'] not in index": 52350470_bldg_6697_op.gml
There are 12 invalid buildings from 8 files
Total files to merge: 261


,height,geometry
0,4.4,"POLYGON ((15072751.097 4117217.254, 15072744.0..."
1,5.2,"POLYGON ((15072760.795 4117204.681, 15072751.4..."
2,11.5,"POLYGON ((15072609.643 4114039.178, 15072607.8..."
3,6.2,"POLYGON ((15072558.583 4113986.948, 15072558.0..."
4,6.3,"POLYGON ((15072645.993 4114000.366, 15072628.9..."
...,...,...
544275,9.0,"POLYGON ((15089601.610 4130609.416, 15089594.1..."
544276,9.1,"POLYGON ((15089727.284 4130200.798, 15089723.3..."
544277,5.6,"POLYGON ((15089513.115 4130506.862, 15089518.6..."
544278,8.9,"POLYGON ((15089905.222 4130629.136, 15089897.1..."


### New York

In [8]:
in_dir = "data/NewYork_2"
out_path = "data/NewYork_2/new_york.feather"
src_crs = "EPSG:4326"

batch_gml_to_feather(in_dir, out_path, mode= "o", log_name= "new_york", src_crs= src_crs)

Total input files: 170
There are 0 invalid buildings from 0 files
Total files to merge: 170


,height,geometry
0,5.73,"POLYGON ((-8209085.418 5566790.279, -8209067.0..."
1,5.73,"POLYGON ((-8209070.390 5566764.459, -8209066.2..."
2,4.38,"POLYGON ((-8209005.825 5566671.511, -8208999.0..."
3,5.73,"POLYGON ((-8209225.236 5566628.949, -8209213.3..."
4,5.73,"POLYGON ((-8209101.448 5566623.003, -8209085.8..."
...,...,...
5716434,4.38,"POLYGON ((-8215328.048 5027787.294, -8215324.3..."
5716435,4.74,"POLYGON ((-8215264.930 5027796.217, -8215262.7..."
5716436,5.17,"POLYGON ((-8215139.829 5027478.327, -8215138.1..."
5716437,5.45,"POLYGON ((-8215386.792 5027729.412, -8215384.6..."


## Examining problem files

In [4]:
with open("logs/tokyo.txt") as f:
    files = f.read()

files = files.split("\n")
problem_gdfs = []

for f in files:
    # Extracts features
    with fiona.open(f, 'r') as src:
        features = list(src)
    
    problem_gdfs.append(gpd.GeoDataFrame.from_features(features))

problem_gdfs = gpd.GeoDataFrame(pd.concat(problem_gdfs)).reset_index(drop= True)
problem_gdfs = problem_gdfs[["geometry", "measuredHeight"]]

problem_gdfs

,geometry,measuredHeight
0,MULTIPOLYGON EMPTY,2.8
1,MULTIPOLYGON EMPTY,3.6
2,MULTIPOLYGON EMPTY,3.6
3,MULTIPOLYGON EMPTY,16.1
4,MULTIPOLYGON EMPTY,9.3
...,...,...
3649,MULTIPOLYGON EMPTY,6.2
3650,MULTIPOLYGON EMPTY,42.2
3651,MULTIPOLYGON EMPTY,10.3
3652,MULTIPOLYGON EMPTY,19.3


Tokyo Dataset is facing empty multipolygons, so this part aims to read it manually without fiona

In [91]:
# Manually read the gml as xml and extract the geometry
doc = xml.dom.minidom.parse("data/13100_tokyo23-ku_2020_citygml_3_2_op/udx/bldg/53394525_bldg_6697_2_op.gml")
# buildings = doc.getElementsByTagName("bldg:lod1Solid") # This is Plateau dataset specific
buildings = doc.getElementsByTagName("bldg:Building") # This is Plateau dataset specific
buildings_coords = [building\
                    .getElementsByTagName("bldg:lod1Solid")[0]\
                    .getElementsByTagName("gml:Polygon")[0]\
                    .getElementsByTagName("gml:posList")[0]\
                    .childNodes[0]\
                    .nodeValue.split(" ") for building in buildings]
# Conversion is done here to preserve the precision due to python rounding shennanigans
transformer = pyproj.Transformer.from_crs("EPSG:6668", "EPSG:3857")
buildings_coords = [
                    [transformer.transform(buildings_coords[bld][i], 
                        buildings_coords[bld][i+1]) for i in range(0, len(buildings_coords[bld]), 3)] 
                    for bld in range(len(buildings_coords))
                    ]
geometry = gpd.GeoSeries([shapely.Polygon(coords) for coords in buildings_coords])

# Manually extract the height
buildings_height = doc.getElementsByTagName("bldg:measuredHeight")
buildings_height = [float(height.childNodes[0].nodeValue) for height in buildings_height]
height_series = pd.DataFrame(buildings_height, columns= ["height"])

gdf = gpd.GeoDataFrame(data= height_series, geometry= geometry)
gdf

,height,geometry
0,47.2,"POLYGON ((15551281.170 4257975.203, 15551281.1..."
1,9.6,"POLYGON ((15550861.584 4257238.269, 15550857.6..."
2,21.3,"POLYGON ((15550987.243 4257966.064, 15550986.8..."
3,18.9,"POLYGON ((15551071.511 4257915.512, 15551077.0..."
4,29.4,"POLYGON ((15550974.838 4257788.037, 15550971.7..."
...,...,...
1375,7.1,"POLYGON ((15550804.951 4257253.915, 15550804.5..."
1376,6.9,"POLYGON ((15550916.926 4257346.304, 15550903.6..."
1377,7.1,"POLYGON ((15550121.593 4257452.333, 15550122.1..."
1378,2.7,"POLYGON ((15550766.039 4257183.125, 15550763.5..."


In [87]:

gdf = gpd.GeoDataFrame(data= height_series, geometry= geometry)
return gdf

[47.2,
 9.6,
 21.3,
 18.9,
 29.4,
 16.6,
 40.2,
 6.8,
 20.3,
 61.9,
 26.8,
 13.2,
 3.3,
 46.6,
 2.8,
 34.5,
 3.0,
 9.3,
 14.0,
 11.9,
 3.0,
 7.1,
 12.5,
 6.1,
 9.9,
 10.3,
 10.6,
 9.9,
 11.5,
 40.7,
 10.4,
 15.9,
 16.2,
 32.3,
 13.8,
 40.8,
 26.3,
 3.0,
 3.9,
 9.1,
 18.5,
 9.8,
 32.7,
 32.2,
 35.5,
 17.0,
 3.5,
 9.9,
 22.3,
 6.6,
 42.6,
 10.1,
 17.7,
 11.1,
 10.0,
 3.0,
 45.6,
 37.6,
 11.6,
 15.7,
 13.2,
 9.7,
 7.0,
 43.4,
 35.2,
 5.4,
 47.5,
 14.3,
 9.0,
 9.5,
 2.8,
 10.9,
 3.0,
 8.2,
 9.3,
 3.1,
 69.0,
 20.0,
 10.1,
 5.4,
 88.4,
 11.1,
 8.7,
 9.6,
 6.7,
 17.8,
 27.0,
 12.5,
 27.9,
 7.7,
 6.6,
 6.8,
 7.4,
 16.3,
 9.4,
 9.6,
 19.3,
 10.4,
 14.0,
 49.1,
 13.1,
 7.2,
 12.2,
 7.3,
 5.0,
 21.9,
 7.8,
 44.5,
 9.7,
 9.8,
 44.7,
 7.4,
 10.1,
 9.8,
 3.5,
 9.4,
 16.8,
 9.5,
 20.5,
 9.8,
 10.0,
 9.8,
 7.3,
 6.8,
 7.7,
 54.1,
 8.2,
 24.4,
 28.6,
 7.9,
 15.8,
 9.9,
 42.7,
 3.6,
 7.3,
 40.2,
 9.3,
 7.7,
 7.4,
 18.0,
 8.3,
 9.8,
 2.5,
 3.0,
 47.6,
 9.8,
 9.4,
 18.6,
 30.3,
 2.4,
 3.0,
 12.0,
 9.6,
 

In [57]:

doc = xml.dom.minidom.parse("data/13100_tokyo23-ku_2020_citygml_3_2_op/udx/bldg/53394525_bldg_6697_2_op.gml")
buildings = doc.getElementsByTagName("bldg:lod1Solid")
buildings_coords = [building.getElementsByTagName("gml:Polygon")[0].getElementsByTagName("gml:posList")[0].childNodes[0].nodeValue.split(" ") for building in buildings]

# buildings_coords
len(buildings_coords)

2201

In [58]:
transformer = pyproj.Transformer.from_crs("EPSG:6668", "EPSG:3857")
# transformer.transform('35.674506226251864', '139.77237738070502')

In [59]:
# from decimal import Decimal
# xy_coords = list(zip([coords[x] for x in range(0, len(coords), 3)], [coords[y] for y in range(1, len(coords), 3)]))
# Reprojection is done at this step to preserve precision as python rounds off high-precision floats
transformer = pyproj.Transformer.from_crs("EPSG:6668", "EPSG:3857")
buildings_coords = [[transformer.transform(buildings_coords[bld][i], buildings_coords[bld][i+1]) for i in range(0, len(buildings_coords[bld]), 3)] for bld in range(len(buildings_coords))]
# xy_coords = np.array(xy_coords).astype(np.longdouble)
buildings_coords


[[(15559389.876985352, 4255925.866332784),
  (15559393.349397551, 4255932.589326772),
  (15559400.81822271, 4255928.6866771),
  (15559397.345439246, 4255921.962081876),
  (15559389.876985352, 4255925.866332784)],
 [(15559445.361986611, 4255488.371743893),
  (15559448.623479664, 4255497.382190577),
  (15559455.865734516, 4255494.675634071),
  (15559452.604236322, 4255485.665191751),
  (15559445.361986611, 4255488.371743893)],
 [(15559503.006922103, 4255220.1972294785),
  (15559499.02899023, 4255222.656258355),
  (15559500.491139771, 4255225.330074535),
  (15559504.471287753, 4255222.867587556),
  (15559503.006922103, 4255220.1972294785)],
 [(15559577.252886912, 4255620.226729103),
  (15559577.324198987, 4255620.448360487),
  (15559565.527820839, 4255624.623220551),
  (15559567.115649411, 4255629.449967565),
  (15559579.507822325, 4255625.064499013),
  (15559579.478734214, 4255624.971844161),
  (15559581.651547153, 4255624.24589609),
  (15559580.024556108, 4255619.301912818),
  (15559577

In [60]:
len(buildings_coords)

2201

In [61]:
geometry = [shapely.Polygon(coords) for coords in buildings_coords]
len(geometry)


2201

In [64]:
gpd.GeoSeries(geometry)

0       POLYGON ((15559389.877 4255925.866, 15559393.3...
1       POLYGON ((15559445.362 4255488.372, 15559448.6...
2       POLYGON ((15559503.007 4255220.197, 15559499.0...
3       POLYGON ((15559577.253 4255620.227, 15559577.3...
4       POLYGON ((15558921.550 4255575.665, 15558916.1...
                              ...                        
2196    POLYGON ((15558350.519 4255925.869, 15558346.8...
2197    POLYGON ((15559357.107 4255984.563, 15559346.8...
2198    POLYGON ((15558793.566 4255884.237, 15558798.4...
2199    POLYGON ((15559535.299 4255649.194, 15559526.1...
2200    POLYGON ((15559080.017 4255751.171, 15559079.5...
Length: 2201, dtype: geometry

In [4]:
with open("logs/osaka.txt") as f:
    files = f.read()

files = files.split("\n")
problem_gdfs = []

for f in files:
    # Extracts features
    with fiona.open(f, 'r') as src:
        features = list(src)
    
    problem_gdfs.append(gpd.GeoDataFrame.from_features(features))

problem_gdfs = gpd.GeoDataFrame(pd.concat(problem_gdfs)).reset_index(drop= True)
# problem_gdfs = problem_gdfs[["geometry", "measuredHeight"]]
problem_gdfs

,geometry,gml_id,建物ID,枝番,prefecture,city,key,codeValue,theme,imageURI,mimeType
0,"MULTIPOLYGON Z (((135.37611 34.64792 1.25830, ...",BLD_4e8cebca-8759-4283-9f2c-099eeac4a7ae,27100-bldg-1,1.0,27.0,27100.0,2.0,2.0,NaN,NaN,NaN
1,"MULTIPOLYGON Z (((135.37730 34.64591 2.22380, ...",BLD_96cc393f-efde-464e-8e90-129c4ccad2ef,27100-bldg-3,1.0,27.0,27100.0,2.0,2.0,NaN,NaN,NaN
2,"MULTIPOLYGON Z (((135.37911 34.64358 3.14470, ...",BLD_80a14177-1694-4eab-b843-7c527b887402,27100-bldg-4,1.0,27.0,27100.0,2.0,2.0,NaN,NaN,NaN
3,"MULTIPOLYGON Z (((135.37907 34.64354 3.28320, ...",BLD_9f6e5948-6968-4192-87aa-458ebc269f55,27100-bldg-5,1.0,27.0,27100.0,2.0,2.0,NaN,NaN,NaN
4,"MULTIPOLYGON Z (((135.37687 34.64791 3.55590, ...",BLD_a3b46c03-5a24-4494-bf90-f07d3738e228,27100-bldg-2,1.0,27.0,27100.0,2.0,2.0,NaN,NaN,NaN
5,None,fme-gen-ece9cd30-11f8-492b-9a2d-ad6153a7f88a,NaN,NaN,NaN,NaN,NaN,NaN,rgbTexture,NaN,NaN
6,None,fme-gen-ebed22fd-125f-45b4-815d-9080d68b9498,NaN,NaN,NaN,NaN,NaN,NaN,rgbTexture,NaN,NaN
7,None,fme-gen-d9f03ada-8ffd-418f-afb2-067ab5012918,NaN,NaN,NaN,NaN,NaN,NaN,rgbTexture,NaN,NaN
8,None,fme-gen-6e1841d6-676a-4d90-b39d-a70ee4691c0f,NaN,NaN,NaN,NaN,NaN,NaN,rgbTexture,NaN,NaN
9,None,fme-gen-0dc34428-8b95-46c5-8dd9-a976f8c3f919,NaN,NaN,NaN,NaN,NaN,NaN,rgbTexture,52350422_bldg_6697_appearance/27100-bldg-32887...,image/jpg


,geometry,measuredHeight


## Divide into grids

### Load files

In [ ]:
# Tokyo


# Osaka

# New York

### Extract Points from datasets

### Get concave hulls that are bounding the dataset

### Divide into grids based on min and max point, then filter out valid grids

## Calculate UMP and export as y

## Download Sentinel and export as X